In [1]:
import numpy as np
from tqdm import tqdm
from itertools import product

%matplotlib inline
import matplotlib.pyplot as plt

In [32]:
def user_print(board):
    length = board.shape[0]
    text = ""
    text += ("\n" + ("-" * 4) * length + "-") + "\n"
    for rows in board:
        text += "| "
        for col in rows:

            if int(col) == Minesweeper.DEFAULT:
                char_to_print = f"{' ':^1}"
            elif int(col) == Minesweeper.BOMB:
                char_to_print = f"{'•':^1}"
            elif int(col) == 0:
                char_to_print = f"{'O':^1}"
            elif int(col) == Minesweeper.LABELBOMB:
                char_to_print = f"{'⚐':^1}"
            else:
                char_to_print = f"{int(col):^1}"

            text += f"{char_to_print} | "

        text += "\n" + ("-" * 4) * length + "-\n"
    print(text)

class Minesweeper:

    DEFAULT = -98  # Empty tiles. (Haven't been chosen yet)
    BOMB = -100  # Bombs that are not yet labeled.
    LABELBOMB = -99  # Bombs the user has labeled.

    def __init__(self, length: int, width: int, bombs: int, custom_board=None):
        self.length = length
        self.width = width
        self.bombs = bombs
        self.moves = 0

        # Create a new board if a custom board is not given.
        if custom_board is None:
            self.board = np.zeros((self.length, self.width)) + Minesweeper.DEFAULT
        else:
            self.board = custom_board

        self.game_running = True


    def valid(self, x: int, y: int):
        return 0 <= x < self.length and 0 <= y < self.width

    def adj(self, x: int, y: int):
        return self.board[max(0, x-1):min(self.length, x+2),
                          max(0, y-1):min(self.width, y+2)]

    def reveal(self, x: int, y: int):
        def _reveal(xi, yi):
            if (not self.valid(xi, yi)
                or self.board[xi, yi] >= 0
                    or self.board[xi, yi] in [Minesweeper.BOMB, Minesweeper.LABELBOMB]):
                return
            # Set the total to the sum of the bombs and the labeled bombs.
            self.board[xi, yi] = (np.sum(self.adj(xi, yi) == Minesweeper.BOMB) +
                                  np.sum(self.adj(xi, yi) == Minesweeper.LABELBOMB))
            # Reveal the neighbors.
            if self.board[xi, yi] == 0:
                _reveal(xi - 1, yi - 1)
                _reveal(xi + 1, yi + 1)
                _reveal(xi + 1, yi - 1)
                _reveal(xi - 1, yi + 1)
                _reveal(xi, yi - 1)
                _reveal(xi, yi + 1)
                _reveal(xi - 1, yi)
                _reveal(xi + 1, yi)

        if not self.valid(x, y) or self.board[x, y] >= 0 or not self.game_running:
            return
        
        self.moves += 1
        
        if self.board[x, y] == Minesweeper.BOMB or self.board[x, y] == Minesweeper.LABELBOMB:
            self.game_running = False
            return
        
        if self.moves == 1:
            probs = np.ones_like(self.board)
            probs[max(0, x-1):min(self.length, x+2),max(0, y-1):min(self.width, y+2)] = 0
            probs[:, :] = 1 / np.count_nonzero(probs)
            probs[max(0, x-1):min(self.length, x+2),max(0, y-1):min(self.width, y+2)] = 0
            probs = probs.flatten()
            
            shape = self.board.shape
            temp = self.board.flatten()
            temp[np.random.choice(temp.shape[0], self.bombs,
                                  replace=False, p=probs)] = Minesweeper.BOMB
            temp = temp.reshape(shape)
            self.board = temp

        _reveal(x, y)
        if self.game_won():
            self.game_running = False

    def mark(self, x: int, y: int):
        if not self.valid(x, y):
            return True
        if self.board[x, y] != Minesweeper.BOMB:
            self.game_running = False
            return False  # Wrong Move! A non bomb tile has been covered.
        self.board[x, y] = Minesweeper.LABELBOMB
        return True

    def user_view(self):
        board = np.copy(self.board)
        board[board == Minesweeper.BOMB] = Minesweeper.DEFAULT
        return board

    def revealed(self):
        return np.argwhere(self.board >= 0)

    def bombs_remaining(self):
        return np.sum(self.board == Minesweeper.BOMB)

    def squares_remaining(self):
        return np.sum((self.board == Minesweeper.DEFAULT))
    
    def game_won(self):
        return self.bombs_remaining() == 0 or self.squares_remaining() == 0

    def print(self):
        text = "  | "
        for i in range(self.length):
            char_to_print = f"{i:^1}"
            text += f"{char_to_print} | "
        text += ("\n  " + ("-" * 4) * self.length + "-") + "\n"
        row_num = 0
        for rows in self.board:
            
            text += f"{row_num} | "
            row_num += 1
            for col in rows:

                if int(col) == Minesweeper.DEFAULT:
                    char_to_print = f"{' ':^1}"
                elif int(col) == Minesweeper.BOMB:
                    char_to_print = f"{'•':^1}"
                elif int(col) == 0:
                    char_to_print = f"{'O':^1}"
                elif int(col) == Minesweeper.LABELBOMB:
                    char_to_print = f"{'⚐':^1}"
                else:
                    char_to_print = f"{int(col):^1}"

                text += f"{char_to_print} | "

            text += "\n  " + ("-" * 4) * self.length + "-\n"
        print(text)

    def user_print(self):
        text = "  | "
        for i in range(self.length):
            char_to_print = f"{i:^1}"
            text += f"{char_to_print} | "
        text += ("\n  " + ("-" * 4) * self.length + "-") + "\n"
        row_num = 0
        board = self.user_view()
        for rows in board:
            text += f"{row_num} | "
            row_num += 1
            for col in rows:

                if int(col) == Minesweeper.DEFAULT:
                    char_to_print = f"{' ':^1}"
                elif int(col) == 0:
                    char_to_print = f"{'O':^1}"
                elif int(col) == Minesweeper.LABELBOMB:
                    char_to_print = f"{'⚐':^1}"
                else:
                    char_to_print = f"{int(col):^1}"

                text += f"{char_to_print} | "

            text += "\n  " + ("-" * 4) * self.length + "-\n"
        print(text)
        
board = Minesweeper(6, 6, 5)
board.print()
board.user_print()

  | 0 | 1 | 2 | 3 | 4 | 5 | 
  -------------------------
0 |   |   |   |   |   |   | 
  -------------------------
1 |   |   |   |   |   |   | 
  -------------------------
2 |   |   |   |   |   |   | 
  -------------------------
3 |   |   |   |   |   |   | 
  -------------------------
4 |   |   |   |   |   |   | 
  -------------------------
5 |   |   |   |   |   |   | 
  -------------------------

  | 0 | 1 | 2 | 3 | 4 | 5 | 
  -------------------------
0 |   |   |   |   |   |   | 
  -------------------------
1 |   |   |   |   |   |   | 
  -------------------------
2 |   |   |   |   |   |   | 
  -------------------------
3 |   |   |   |   |   |   | 
  -------------------------
4 |   |   |   |   |   |   | 
  -------------------------
5 |   |   |   |   |   |   | 
  -------------------------



In [33]:
%%time

board = Minesweeper(7, 7, 10)
board.reveal(3, 3)
board.print()

def is_valid(b):
    l, w = b.shape
    for (x, y) in np.argwhere((b > 0)):
        num_listed_bombs = b[x, y]
        num_neighbor_bombs = np.sum((b[max(0, x-1):min(l, x+2),
                                       max(0, y-1):min(w, y+2)] == Minesweeper.LABELBOMB))
        
        if num_listed_bombs != num_neighbor_bombs:
            return False
    return True

def build_next_layer(board):
    valid = []
    
    b = np.zeros_like(board.board)
    l, w = b.shape

    revealed_squares = board.revealed()

    for (x, y) in revealed_squares:
        b[max(0, x-1):min(l, x+2),max(0, y-1):min(w, y+2)] = 1
    b[revealed_squares[:, 0], revealed_squares[:, 1]] = 0
    
    indicies_for_bombs = np.argwhere((b == 1))
    num_squares = indicies_for_bombs.shape[0]
    
    possible_bombs = list(product([Minesweeper.LABELBOMB, Minesweeper.DEFAULT], repeat=num_squares))
    for bomblist in possible_bombs:
        b = board.user_view()
        b[indicies_for_bombs[:, 0], indicies_for_bombs[:, 1]] = bomblist
        if(is_valid(b)):
            valid.append(bomblist)
    return np.array(valid), indicies_for_bombs

valid_bombs, indicies_for_bombs = build_next_layer(board)        

b = board.user_view()
# b[indicies_for_bombs[:, 0], indicies_for_bombs[:, 1]] = valid_bombs[0]
print(len(valid_bombs))
user_print(b)

  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 
  -----------------------------
0 |   | • | 1 | O | O | 1 | • | 
  -----------------------------
1 | 1 | 1 | 1 | O | O | 2 |   | 
  -----------------------------
2 | O | O | O | O | 1 | 2 | • | 
  -----------------------------
3 | O | O | O | O | 1 | • |   | 
  -----------------------------
4 | 1 | 2 | 1 | 1 | 1 |   | • | 
  -----------------------------
5 | • |   | • |   |   |   | • | 
  -----------------------------
6 | • |   | • |   |   |   |   | 
  -----------------------------

1

-----------------------------
|   |   | 1 | O | O | 1 |   | 
-----------------------------
| 1 | 1 | 1 | O | O | 2 |   | 
-----------------------------
| O | O | O | O | 1 | 2 |   | 
-----------------------------
| O | O | O | O | 1 |   |   | 
-----------------------------
| 1 | 2 | 1 | 1 | 1 |   |   | 
-----------------------------
|   |   |   |   |   |   |   | 
-----------------------------
|   |   |   |   |   |   |   | 
-----------------------------

CPU times: user 29

In [37]:
def is_valid(b):
    l, w = b.shape
    for (x, y) in np.argwhere((b > 0)):
        num_listed_bombs = b[x, y]
        num_neighbor_bombs = np.sum((b[max(0, x-1):min(l, x+2),
                                       max(0, y-1):min(w, y+2)] == Minesweeper.LABELBOMB))
        
        if num_listed_bombs != num_neighbor_bombs:
            return False
    return True

def build_next_layer(board):
    valid = []
    
    b = np.zeros_like(board.board)
    l, w = b.shape

    revealed_squares = board.revealed()

    for (x, y) in revealed_squares:
        b[max(0, x-1):min(l, x+2),max(0, y-1):min(w, y+2)] = 1
    b[revealed_squares[:, 0], revealed_squares[:, 1]] = 0
    
    indicies_for_bombs = np.argwhere((b == 1))
    num_squares = indicies_for_bombs.shape[0]
    
    possible_bombs = list(product([Minesweeper.LABELBOMB, Minesweeper.DEFAULT], repeat=num_squares))
    for bomblist in tqdm(possible_bombs, ncols=90):
        b = board.user_view()
        b[indicies_for_bombs[:, 0], indicies_for_bombs[:, 1]] = bomblist
        if(is_valid(b)):
            valid.append(bomblist)
    return np.array(valid), indicies_for_bombs

def get_guess(board):
    valid_squares, indicies = build_next_layer(board)
    valid_squares[valid_squares == -99] = 0
    valid_squares[valid_squares == -98] = 1
    probs = np.sum(valid_squares, axis=0) / valid_squares.shape[0]
    guess = indicies[np.argwhere((probs == np.max(probs)))]
    return guess, np.max(probs), valid_squares.shape[0]

board = Minesweeper(7, 7, 10)
board.reveal(3, 3)
board.print()
guess, prob, guesses = get_guess(board)
print(guess, prob, guesses)

  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 
  -----------------------------
0 | • |   |   |   |   | 1 | O | 
  -----------------------------
1 |   | • |   |   | • | 1 | O | 
  -----------------------------
2 | • | • | 2 | 1 | 1 | 1 | O | 
  -----------------------------
3 | • |   | 1 | O | O | O | O | 
  -----------------------------
4 |   |   | 1 | 1 | 1 | 1 | 1 | 
  -----------------------------
5 |   |   |   | • |   |   | • | 
  -----------------------------
6 |   |   |   | • |   | • |   | 
  -----------------------------



100%|████████████████████████████████████████████| 16384/16384 [00:00<00:00, 51754.23it/s]

[[[0 4]]

 [[1 2]]

 [[1 3]]

 [[3 1]]

 [[4 1]]

 [[5 1]]

 [[5 4]]] 1.0 2


In [38]:
board = Minesweeper(10, 10, 20)
board.reveal(2, 2)

while board.game_running:
    board.print()
    guess, prob, guesses = get_guess(board)
    guess = guess[0]
    print(guess, prob, guesses)
    if prob == 1.0:
        for (x, y) in guess:
            board.reveal(x, y)
    else:
        guess = guess[0]
        board.reveal(guess[0], guess[1])
    
if board.game_won():
    print("Game Won!")
else:
    print("Game Lost :(")

  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 
  -----------------------------------------
0 | O | 1 | • | 1 | O | O | O | O | O | O | 
  -----------------------------------------
1 | O | 1 | 1 | 1 | O | 1 | 1 | 1 | O | O | 
  -----------------------------------------
2 | O | O | O | O | O | 1 | • | 1 | O | O | 
  -----------------------------------------
3 | O | 1 | 2 | 3 | 2 | 2 |   | 2 | 2 | 1 | 
  -----------------------------------------
4 | 1 | 2 | • | • | • |   |   | • |   | • | 
  -----------------------------------------
5 |   | • |   |   |   |   | • | • |   |   | 
  -----------------------------------------
6 |   |   |   | • |   |   |   | • |   |   | 
  -----------------------------------------
7 | • | • |   |   |   | • |   | • | • |   | 
  -----------------------------------------
8 |   |   |   |   | • |   |   |   |   |   | 
  -----------------------------------------
9 | • |   |   |   |   |   |   |   | • |   | 
  -----------------------------------------



100%|████████████████████████████████████████████| 16384/16384 [00:00<00:00, 34029.25it/s]


[[3 6]] 1.0 2
  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 
  -----------------------------------------
0 | O | 1 | • | 1 | O | O | O | O | O | O | 
  -----------------------------------------
1 | O | 1 | 1 | 1 | O | 1 | 1 | 1 | O | O | 
  -----------------------------------------
2 | O | O | O | O | O | 1 | • | 1 | O | O | 
  -----------------------------------------
3 | O | 1 | 2 | 3 | 2 | 2 | 2 | 2 | 2 | 1 | 
  -----------------------------------------
4 | 1 | 2 | • | • | • |   |   | • |   | • | 
  -----------------------------------------
5 |   | • |   |   |   |   | • | • |   |   | 
  -----------------------------------------
6 |   |   |   | • |   |   |   | • |   |   | 
  -----------------------------------------
7 | • | • |   |   |   | • |   | • | • |   | 
  -----------------------------------------
8 |   |   |   |   | • |   |   |   |   |   | 
  -----------------------------------------
9 | • |   |   |   |   |   |   |   | • |   | 
  -----------------------------------------



100%|██████████████████████████████████████████████| 8192/8192 [00:00<00:00, 34584.12it/s]


[[4 5]] 1.0 2
  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 
  -----------------------------------------
0 | O | 1 | • | 1 | O | O | O | O | O | O | 
  -----------------------------------------
1 | O | 1 | 1 | 1 | O | 1 | 1 | 1 | O | O | 
  -----------------------------------------
2 | O | O | O | O | O | 1 | • | 1 | O | O | 
  -----------------------------------------
3 | O | 1 | 2 | 3 | 2 | 2 | 2 | 2 | 2 | 1 | 
  -----------------------------------------
4 | 1 | 2 | • | • | • | 2 |   | • |   | • | 
  -----------------------------------------
5 |   | • |   |   |   |   | • | • |   |   | 
  -----------------------------------------
6 |   |   |   | • |   |   |   | • |   |   | 
  -----------------------------------------
7 | • | • |   |   |   | • |   | • | • |   | 
  -----------------------------------------
8 |   |   |   |   | • |   |   |   |   |   | 
  -----------------------------------------
9 | • |   |   |   |   |   |   |   | • |   | 
  -----------------------------------------



...

[[6 4]] 0.8571428571428571 14
  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 
  -----------------------------------------
0 | O | 1 | • | 1 | O | O | O | O | O | O | 
  -----------------------------------------
1 | O | 1 | 1 | 1 | O | 1 | 1 | 1 | O | O | 
  -----------------------------------------
2 | O | O | O | O | O | 1 | • | 1 | O | O | 
  -----------------------------------------
3 | O | 1 | 2 | 3 | 2 | 2 | 2 | 2 | 2 | 1 | 
  -----------------------------------------
4 | 1 | 2 | • | • | • | 2 | 3 | • | 3 | • | 
  -----------------------------------------
5 |   | • | 4 | 4 | 3 |   | • | • | 4 | 1 | 
  -----------------------------------------
6 |   |   |   | • | 2 |   |   | • | 4 | 1 | 
  -----------------------------------------
7 | • | • |   |   |   | • |   | • | • |   | 
  -----------------------------------------
8 |   |   |   |   | • |   |   |   |   |   | 
  -----------------------------------------
9 | • |   |   |   |   |   |   |   | • |   | 
  ---------------------------------

100%|████████████████████████████████████████| 8388608/8388608 [03:59<00:00, 35088.77it/s]


[[5 5]] 0.6666666666666666 36
  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 
  -----------------------------------------
0 | O | 1 | • | 1 | O | O | O | O | O | O | 
  -----------------------------------------
1 | O | 1 | 1 | 1 | O | 1 | 1 | 1 | O | O | 
  -----------------------------------------
2 | O | O | O | O | O | 1 | • | 1 | O | O | 
  -----------------------------------------
3 | O | 1 | 2 | 3 | 2 | 2 | 2 | 2 | 2 | 1 | 
  -----------------------------------------
4 | 1 | 2 | • | • | • | 2 | 3 | • | 3 | • | 
  -----------------------------------------
5 |   | • | 4 | 4 | 3 | 2 | • | • | 4 | 1 | 
  -----------------------------------------
6 |   |   |   | • | 2 |   |   | • | 4 | 1 | 
  -----------------------------------------
7 | • | • |   |   |   | • |   | • | • |   | 
  -----------------------------------------
8 |   |   |   |   | • |   |   |   |   |   | 
  -----------------------------------------
9 | • |   |   |   |   |   |   |   | • |   | 
  ---------------------------------

 32%|████████████▉                           | 2721977/8388608 [02:22<04:56, 19104.13it/s]


KeyboardInterrupt: 